# Creating the Tools with Standalone Callbacks 
Goal: create the tools so that people can use them as a python script

Steps to complete the main tool:
1. set up changeable channels with the most basic barebone example
2. link the two plots together
3. link to the frequency slider (mind the javacall back: callback_policy="mouseup")
4. set up histogram and bin changer
5. link histogram data. make sure it can be updated
6. fix minor details such as plot labels etc.

In [1]:
import sys
# locate your spectralCV so we have scv_funcs to use
sys.path.append('/Users/ldliao/Research/Projects/spectralCV/')
sys.path.append('/Users/ldliao/Research/Projects/spectralCV/notebooks_visualization/')

In [2]:
import os
# the only lines you need to change to inspect different saved data
path = '/Users/ldliao/Research/Projects/spectralCV/results/kjm_digits/bp/'
os.chdir(path)
import glob
from scv_funcs import lfpca

In [14]:
# the final product to import the visualization
import lfpca_vis

Loading BokehJS ...

In [3]:
# collect all the names of .npz files in the folder
lfpca_all_names = glob.glob("*.npz")
lfpca_all_names.sort()

# loading the npz files 
lfpca_all = {}
for ind, lf in enumerate(lfpca_all_names):
    lfpca_all[lf[:-4]] = lfpca.lfpca_load_spec(lf)

# initialize with the first lfpca object
lf = lfpca_all[lfpca_all_names[0][:-4]]

In [16]:
# ignore bokeh warnings
import warnings
warnings.filterwarnings('ignore')

lfpca_vis.plot_vis(lf=lf)

interactive(children=(IntSlider(value=10, description='f', max=199, min=1), IntSlider(value=1, description='ch…

### Imports

In [5]:
# imports
import numpy as np
import scipy as sp
from scipy.stats import expon
import glob

# import neurodsp as ndsp
# from scv_funcs import lfpca
import warnings
warnings.filterwarnings('ignore')

In [10]:
# bokeh imports
import bokeh
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, Slider, Span
from bokeh.models.widgets import Select, Slider
from ipywidgets import interact

output_notebook()

Loading BokehJS ...

### Defaults

In [11]:
# setting up the defaults
chan = 0
select_freq = 10
select_bin = 20

freq_vals = lf.f_axis[1:]
psd_vals = lf.psd[chan].T[1:]
scv_vals = lf.scv[chan].T[1:]

source = ColumnDataSource(data=dict(freq_vals=freq_vals, 
                                    psd_vals=psd_vals, 
                                    scv_vals=scv_vals))
# grabbing channel count from psd
chan_count, freq = lf.psd.shape
DEFAULT_TICKERS = list(map(str, range(chan_count)))

### Sliders and updates

In [12]:
# create interact
def update_spct_hist(f=10, channel=0, numbins=20):
    plot_chan = int(channel)
    plot_freq = np.where(lf.f_axis==f)[0][0]
    y, x = np.histogram(lf.spg[plot_chan,plot_freq,:], bins=numbins, density=True)
    
    # create a column data source for the plots to share
    data_source = {
                   'freq_vals': lf.f_axis[1:],
                   'psd_vals': lf.psd[channel].T[1:],
                   'scv_vals': lf.scv[channel].T[1:]
                   }   
    source.data = data_source
    vline_psd.location = f
    vline_scv.location = f
    # update histogram with data from frequency f
    hist_plot.data_source.data['left'] = x[:-1]
    hist_plot.data_source.data['right'] = x[1:]
    hist_plot.data_source.data['top'] = y    
    # update fitted
    rv = expon(scale=sp.stats.expon.fit(lf.spg[plot_chan,plot_freq,:],floc=0)[1])
    fit_plot.data_source.data['x'] = x
    fit_plot.data_source.data['y'] = rv.pdf(x)
    
    hist_fig.title.text = 'Freq = %.1fHz, p-value = %.4f'%(f, lf.ks_pvals[int(channel), f])
    push_notebook()

### Set up the plots

In [17]:
# set up histogram
y, x = np.histogram(lf.spg[0,10,:], bins=20, density=True)
hist_fig = figure(plot_height=300, plot_width=300, x_axis_label='Power', y_axis_label='Probability')
hist_plot = hist_fig.quad(top=y, bottom=0, left=x[:-1], right=x[1:], fill_color='#295B99', line_color="#033649")

fit_plot = hist_fig.line(x[:-1],y, line_width=8,alpha=0.7,line_color="#D53B54",legend='Fit PDF')

# set up psd plot
psd_plot = figure(title='PSD', x_axis_type='log', y_axis_type='log', plot_width=300, plot_height=300)
psd_plot.legend.location = 'top_left'
psd_plot.xaxis.axis_label = 'Frequency (Hz)'
psd_plot.yaxis.axis_label = 'Power/Frequency (dB/Hz)'
psd_plot.grid.grid_line_alpha=0.3
psd_plot.line('freq_vals', 'psd_vals', source=source)    

# set up scv plot
scv_plot = figure(title='SCV', x_axis_type='log', y_axis_type='log', plot_width=300, plot_height=300)
scv_plot.legend.location='top_left'
scv_plot.xaxis.axis_label = 'Frequency (Hz)'
scv_plot.yaxis.axis_label = '(Unitless)'
scv_plot.grid.grid_line_alpha=0.3
fit_line = bokeh.models.glyphs.Line(x='freq_vals', y=1, line_width=5, line_alpha=0.5, line_color='darkgrey')
scv_plot.add_glyph(source, fit_line)
scv_plot.line('freq_vals', 'scv_vals', source=source, color='navy')

# add in frequency slider vertical lines
vline_psd = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
vline_scv = Span(location=select_freq, dimension='height', line_color='red', line_dash='dashed', line_width=3)
psd_plot.add_layout(vline_psd)
scv_plot.add_layout(vline_scv)

# set up layout and interact toola
layout = column(row(psd_plot, scv_plot, hist_fig))
show(layout, notebook_handle=True)

import warnings
warnings.filterwarnings('ignore')
interact(update_spct_hist, f=(1,199), numbins=range(10,55,5), channel=(1,len(DEFAULT_TICKERS)-1))

interactive(children=(IntSlider(value=10, description='f', max=199, min=1), IntSlider(value=1, description='ch…

<function __main__.update_spct_hist(f=10, channel=0, numbins=20)>